<img style="float: left;" src="http://third.datacastle.cn/pkbigdata/master.other.img/8ef429f9-2032-47b8-bcc4-818fa9e41a25.png" width="50%">

In [11]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
op_train = pd.read_csv('../data/operation_train_new.csv')
tran_train = pd.read_csv('../data/transaction_train_new.csv')
tag_train = pd.read_csv('../data/tag_train_new.csv')

In [3]:
print(op_train.shape)
print(tran_train.shape)

(1460843, 20)
(264654, 27)


### 1.数据探索

In [4]:
def find_different_uid(op_uids, tran_uids):
    op_diff_uids = []
    for uid in op_uids:
        if uid not in tran_uids:
            op_diff_uids.append(uid)
        else:
            tran_uids.remove(uid)
    return op_diff_uids, tran_uids

def find_same_uid(src1_uids, src2_uids):
    same_uids = [uid for uid in src1_uids if uid in src2_uids]
    return same_uids

In [5]:
# 处理时间字符串
op_train['time'] = op_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + op_train['time']
op_train['timestamp'] = op_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

tran_train['time'] = tran_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + tran_train['time']
tran_train['timestamp'] = tran_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

In [31]:
# 根据UID进行排序 再根据timestamp进行排序 
op_train = op_train.sort_values(by=['UID', 'timestamp'],ascending=True)
tran_train = tran_train.sort_values(by=['UID', 'timestamp'],ascending=True)
tag_train = tag_train.sort_values(by=['UID'], ascending=True)
op_train_gb = op_train.groupby('UID', as_index=False)
tran_train_gb = tran_train.groupby('UID', as_index=False)

In [ ]:
# (该方法已被废弃)
# 根据UID进行分组，再根据timestamp进行排序 
op_train_gb = op_train.groupby('UID').apply(lambda x:x.sort_values(by='timestamp', ascending=True))
tran_train_gb = tran_train.groupby('UID').apply(lambda x:x.sort_values(by='timestamp', ascending=True))
# tag_train_gb = tag_train.groupby('UID', as_index=True)
op_train_gb = op_train_gb.groupby('UID')
tran_train_gb = tran_train_gb.groupby('UID')

#### 1.1对op进行分析

In [32]:
#  获取op和tran各自的uid
op_train_uids = [uid for uid, item in op_train_gb.groups.items()]
tran_train_uids = [uid for uid, item in tran_train_gb.groups.items()]

# 获取op和tran各自的tag
op_tag = tag_train[tag_train['UID'].isin(op_train['UID'])]
tran_tag = tag_train[tag_train['UID'].isin(tran_train['UID'])]

In [45]:
# 将op数据中的作假者与正常人分离开
op_train_gb_cnt = op_train_gb.count()
op_train_fake_stat = op_train_gb_cnt[op_train_gb_cnt.index.isin(op_tag['UID'][op_tag['Tag'] == 1])]
op_train_normal_stat = op_train_gb_cnt[op_train_gb_cnt.index.isin(op_tag['UID'][op_tag['Tag'] == 0])]

op_train_fake_gp = op_train[op_train['UID'].isin(op_tag['UID'][op_tag['Tag'] == 1])].groupby('UID', as_index=False)
op_train_normal_gp = op_train[op_train['UID'].isin(op_tag['UID'][op_tag['Tag'] == 0])].groupby('UID', as_index=False)

In [51]:
print(len(op_train_fake_gp))
op_train_fake_gp.head()

4277


,index,UID,day,mode,success,time,os,version,device1,device2,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
0,0,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
1,1,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
2,2,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
3,3,10000,13,270ca3ca8f0d1126,1.0,2018-08-13 14:22:59,103,7.0.5,aca4977fbe8741e1,IPHONE 5,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
4,4,10000,26,c8741ce15ceac2a4,1.0,2018-08-26 12:23:48,103,7.0.7,aca4977fbe8741e1,IPHONE 5,...,deb68f7028682008,NaN,NaN,34c715882de1a508,NaN,NaN,NaN,1e3ea9498c461cbf,NaN,1.535257e+09
397,397,10019,1,c8741ce15ceac2a4,1.0,2018-08-01 21:58:47,103,7.0.5,aca4977fbe8741e1,IPHONE 5C,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
398,398,10019,1,c8741ce15ceac2a4,1.0,2018-08-01 21:58:47,103,7.0.5,aca4977fbe8741e1,IPHONE 5C,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
399,399,10019,1,d25caee90b27fa9b,1.0,2018-08-01 21:59:45,103,7.0.5,aca4977fbe8741e1,NaN,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
400,400,10019,1,d25caee90b27fa9b,1.0,2018-08-01 22:00:05,103,7.0.5,aca4977fbe8741e1,NaN,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
401,401,10019,6,c8741ce15ceac2a4,1.0,2018-08-06 18:03:36,103,7.0.5,aca4977fbe8741e1,IPHONE 5C,...,d9cf6be4f1fb9dfa,NaN,NaN,065928d73f7c9597,NaN,NaN,NaN,62bfad343395c711,NaN,1.533550e+09


In [47]:
print(len(op_train_normal_gp))
op_train_normal_gp.head()

25451


,index,UID,day,mode,success,time,os,version,device1,device2,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
9,9,10001,2,c8741ce15ceac2a4,1.0,2018-08-02 08:30:52,102,6.6.2,09baf2f39bc3dc86,HUAWEI GRA-CL00,...,NaN,NaN,978e38f2d0431a30,2c1958d001a993d9,NaN,NaN,wsgb,bd5e6a16e61e61f7,NaN,1.533170e+09
10,10,10001,2,c8741ce15ceac2a4,1.0,2018-08-02 08:30:54,102,6.6.2,09baf2f39bc3dc86,HUAWEI GRA-CL00,...,NaN,NaN,978e38f2d0431a30,2c1958d001a993d9,NaN,NaN,wsgb,bd5e6a16e61e61f7,NaN,1.533170e+09
11,11,10001,2,79f8b86938180c3c,NaN,2018-08-02 08:31:16,200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.533170e+09
12,12,10001,2,79f8b86938180c3c,NaN,2018-08-02 08:31:17,200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.533170e+09
13,13,10001,2,d25caee90b27fa9b,1.0,2018-08-02 08:31:23,102,6.6.2,09baf2f39bc3dc86,NaN,...,NaN,NaN,978e38f2d0431a30,2c1958d001a993d9,NaN,NaN,wsgb,bd5e6a16e61e61f7,NaN,1.533170e+09
76,76,10002,29,1c341176507fbd9b,1.0,2018-08-29 09:52:11,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,...,NaN,NaN,31c72f5ec93149ee,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
77,77,10002,29,9c69742a831d6214,1.0,2018-08-29 09:52:11,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,...,NaN,2af8f4efc0bfdf01,NaN,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
78,78,10002,29,8c7ce5ff939e66d1,1.0,2018-08-29 09:52:32,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,...,NaN,a8dc52f65085212e,NaN,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
79,79,10002,29,ac63e881c4e19402,1.0,2018-08-29 09:52:32,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,...,NaN,a8dc52f65085212e,NaN,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
80,80,10002,29,c8741ce15ceac2a4,1.0,2018-08-29 09:53:10,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,...,NaN,NaN,31c72f5ec93149ee,7716777f90955a08,NaN,NaN,wmgu,4dc6f70900fadcab,NaN,1.535508e+09


In [50]:
op_feture_columns = ['day', 'mode', 'success', 'time', 'os', 'version',
                     'device1', 'device2', 'device_code1', 'device_code2', 'device_code3',
                     'mac1', 'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub',
                     'timestamp']

Index(['index', 'UID', 'day', 'mode', 'success', 'time', 'os', 'version',
       'device1', 'device2', 'device_code1', 'device_code2', 'device_code3',
       'mac1', 'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub',
       'timestamp'],
      dtype='object')